In [1]:
import pandas as pd
import optuna
import numpy as np
import json
import pickle
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

In [2]:
# NumPy 랜덤 시드 고정
np.random.seed(2147483648)

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [4]:
def preprocess_data(df):
    # "알 수 없음"을 최빈값으로 대체
    most_frequent_age = df[df["시술 당시 나이"] != "알 수 없음"]["시술 당시 나이"].mode()[0]
    df["시술 당시 나이"] = df["시술 당시 나이"].replace("알 수 없음", most_frequent_age)
    
    # 특정 시술 유형에서 종류가 많아 묶어주기
    df["특정 시술 유형"] = df["특정 시술 유형"].fillna("Unknown")
    
    # 데이터가 50개 미만인 시술 유형을 '기타'로 변경
    procedure_counts = df["특정 시술 유형"].value_counts()
    low_count_procedures = procedure_counts[procedure_counts < 50].index.tolist()
    df["특정 시술 유형"] = df["특정 시술 유형"].apply(lambda x: "기타" if x in low_count_procedures else x)
    
    # "BLASTOCYST"가 포함되면 "BLASTOCYST"로 변경 (단, "기타"로 변경된 데이터는 제외)
    df["특정 시술 유형"] = df["특정 시술 유형"].apply(lambda x: "BLASTOCYST" if "BLASTOCYST" in x and x != "기타" else x)
    
    # "IVF:ICSI"와 "ICSI:IVF"를 "ICSI:IVF"로 통일
    df["특정 시술 유형"] = df["특정 시술 유형"].replace({"IVF:ICSI": "ICSI:IVF"})
    
    # "AH"가 포함된 것은 "AH"로 변경
    df["특정 시술 유형"] = df["특정 시술 유형"].apply(lambda x: "AH" if "AH" in x else x)
    
    # ":Unknown"으로 끝나는 것은 ":Unknown"으로 변경
    df["특정 시술 유형"] = df["특정 시술 유형"].apply(lambda x: ":Unknown" if x.endswith(":Unknown") else x)
    
    # "ICI"는 "IUI"로 변경
    df["특정 시술 유형"] = df["특정 시술 유형"].replace({"ICI": "IUI"})
    
    # "IVF:IVF"와 "ICSI:ICSI"는 "위험"으로 변경
    df["특정 시술 유형"] = df["특정 시술 유형"].replace({"IVF:IVF": "위험", "ICSI:ICSI": "위험"})
    
    # 새로운 파생 변수 생성
    df['해동 난자 활용도'] = df['해동 난자 수'] / (df['해동 난자 수'] + df['수집된 신선 난자 수'])
    df['신선 난자 활용도'] = df['수집된 신선 난자 수'] / (df['수집된 신선 난자 수'] + df['해동 난자 수'])
    df['배아 생성 성공률'] = df['총 생성 배아 수'] / df['혼합된 난자 수']
    df['이식된 배아 비율'] = df['이식된 배아 수'] / df['총 생성 배아 수']
    df['저장된 배아 비율'] = df['저장된 배아 수'] / df['총 생성 배아 수']
    df['미세주입 배아 이식 비율'] = df['미세주입 배아 이식 수'] / df['총 생성 배아 수']
    df['해동 배아 비율'] = df['해동된 배아 수'] / df['총 생성 배아 수']
    df['신선-해동 난자 비율 차이'] = df['신선 난자 활용도'] - df['해동 난자 활용도']
    df['배아 활용도'] = (df['이식된 배아 수'] + df['저장된 배아 수']) / df['총 생성 배아 수']
    df['배아 보존율'] = df['저장된 배아 수'] / (df['이식된 배아 수'] + df['저장된 배아 수'])
    df['배아 손실률'] = 1 - df['배아 활용도']
    df['시험관 난자 수'] = df['혼합된 난자 수'] - df['미세주입된 난자 수']
    df['시험관 배아 생성 성공률'] = (df['총 생성 배아 수'] - df['미세주입에서 생성된 배아 수']) / df['시험관 난자 수']
    df['미세주입 배아 생성 성공률'] = df['미세주입에서 생성된 배아 수'] / df['미세주입된 난자 수']
    
    # NaN 및 무한대 값 처리
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    
    #-----------------------------------------------------------------------------------
    # 실험1
    df["마지막 임신 후 경과 연수"] = df["임신 시도 또는 마지막 임신 경과 연수"].astype(float)
    df["난자 채취-이식 간격"] = df["배아 이식 경과일"].astype(float) - df["난자 채취 경과일"].astype(float)
    df["배아 해동-이식 간격"] = df["배아 이식 경과일"].astype(float) - df["배아 해동 경과일"].astype(float)
    
    # 실험2
    df["ICSI 여부"] = df["특정 시술 유형"].apply(lambda x: 1 if "ICSI" in x else 0)
    df["AH 여부"] = df["특정 시술 유형"].apply(lambda x: 1 if "AH" in x else 0)
    df["시술 복합 유형"] = df["특정 시술 유형"].apply(lambda x: "ICSI+AH" if "ICSI" in x and "AH" in x else x)
    
    # 실험 3
    df["배아 생존율"] = df["이식된 배아 수"].astype(float) / df["해동된 배아 수"].astype(float)
    df["신선 vs 해동 배아 비율"] = df["신선 난자 활용도"] / (df["해동 난자 활용도"] + 1e-6)
    df["난자 대비 배아 성공률"] = df["총 생성 배아 수"].astype(float) / df["수집된 신선 난자 수"].astype(float)
    
    
    #-----------------------------------------------------------------------------------
    # 예외처리할 파생변수 리스트
    exception_columns = [
        '시험관 난자 수', '시험관 배아 생성 성공률', '미세주입 배아 생성 성공률', '배아 활용도',
        '해동 난자 활용도', '신선 난자 활용도', '배아 보존율', '배아 손실률', '배아 생성 성공률',
        '이식된 배아 비율', '저장된 배아 비율', '미세주입 배아 이식 비율', '해동 배아 비율',
        '신선-해동 난자 비율 차이', '불임 원인 코드', '총 불임 원인 수', '남성 불임 원인 수',
        '남성 불임 원인 비율', '여성 불임 원인 수', '여성 불임 원인 비율', '부부 불임 원인 수',
        '부부 불임 원인 비율', '불명확 불임 여부', '불임 원인 PC1', '불임 원인 PC2', '불임 원인 PC3'
    ]
    
    # 문자열로 변환할 컬럼 리스트
    columns_to_convert = [col for col in df.columns if col not in exception_columns]
    df[columns_to_convert] = df[columns_to_convert].astype(str)
    
    return df

# train_df와 test_df 모두 전처리 적용
train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)

In [5]:
# ✅ 종속 변수(Y) 설정
Y = train_df['임신 성공 여부']
X = train_df.drop(columns=['임신 성공 여부'])  # Y를 제외한 모든 데이터

X = X.drop(columns=['ID'])

# ✅ 범주형 컬럼 리스트 생성 (문자열 타입인 컬럼들)
cat_features = X.select_dtypes(include=['object']).columns.tolist()

In [6]:
# ✅ 하이퍼파라미터 설정 (기본값 사용)
default_params = {
    "iterations": 1550,
    "learning_rate": 0.024596749414760054,
    "depth": 7,
    "l2_leaf_reg": 3,
    "subsample": 0.9879384809521996,
    "scale_pos_weight": 2.4420905862453086,
    'random_seed': 42,
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'verbose': 50,
    'early_stopping_rounds': 50  # ✅ 얼리 스토핑 추가
}


In [7]:
# ✅ 최종 모델 학습 (전체 Train 데이터 사용)
model_test = CatBoostClassifier(**default_params)
model_test.fit(X, Y, cat_features=cat_features)

0:	total: 346ms	remaining: 8m 56s
50:	total: 7.65s	remaining: 3m 44s
100:	total: 14.5s	remaining: 3m 28s
150:	total: 21.4s	remaining: 3m 18s
200:	total: 28s	remaining: 3m 7s
250:	total: 34.5s	remaining: 2m 58s
300:	total: 41s	remaining: 2m 50s
350:	total: 47.4s	remaining: 2m 41s
400:	total: 54s	remaining: 2m 34s
450:	total: 1m	remaining: 2m 27s
500:	total: 1m 7s	remaining: 2m 21s
550:	total: 1m 14s	remaining: 2m 14s
600:	total: 1m 21s	remaining: 2m 8s
650:	total: 1m 27s	remaining: 2m 1s
700:	total: 1m 34s	remaining: 1m 54s
750:	total: 1m 41s	remaining: 1m 47s
800:	total: 1m 48s	remaining: 1m 41s
850:	total: 1m 54s	remaining: 1m 34s
900:	total: 2m 1s	remaining: 1m 27s
950:	total: 2m 8s	remaining: 1m 20s
1000:	total: 2m 14s	remaining: 1m 13s
1050:	total: 2m 21s	remaining: 1m 7s
1100:	total: 2m 28s	remaining: 1m
1150:	total: 2m 34s	remaining: 53.7s
1200:	total: 2m 41s	remaining: 47s
1250:	total: 2m 48s	remaining: 40.3s
1300:	total: 2m 55s	remaining: 33.5s
1350:	total: 3m 2s	remaining: 26.

In [8]:
# 학습 데이터와 동일한 컬럼 리스트 사용
features = model_test.feature_names_  # 모델 학습에 사용된 피처 이름 가져오기
test_features = test_df[features]  # 테스트 데이터에서 동일한 컬럼만 선택

# 예측 수행
result_df = test_df[['ID']].copy()  # ID 컬럼 유지
result_df["probability"] = model_test.predict_proba(test_features)[:, 1]  # 양성 클래스(1)의 확률값 사용


# 'DI' 그룹의 확률값을 0.9배 조정 
result_df.loc[test_df['시술 유형'] == 'DI', 'probability'] *= 0.94

In [ ]:
# 예측 결과 저장
#result_df.to_csv("submission.csv", index=False, encoding='utf-8')

In [10]:
result_df

,ID,probability
0,TEST_00000,0.001977
1,TEST_00001,0.002743
2,TEST_00002,0.314248
3,TEST_00003,0.217459
4,TEST_00004,0.711351
...,...,...
90062,TEST_90062,0.001965
90063,TEST_90063,0.528827
90064,TEST_90064,0.637439
90065,TEST_90065,0.400186
